In [1]:
import findspark
findspark.init('/home/fede/spark-2.1.0-bin-hadoop2.7')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [4]:
file_path = '/home/fede/SMSSpamCollection'
data = spark.read.csv(file_path, inferSchema=True, sep='\t')

In [5]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [6]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1','text')

In [7]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import length

In [10]:
data = data.withColumn('length', length(data['text']))

In [11]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [14]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [15]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover,
                               CountVectorizer,IDF,StringIndexer)

In [18]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol = 'stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol ='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [19]:
from pyspark.ml.feature import VectorAssembler

In [33]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')

In [34]:
from pyspark.ml.classification import NaiveBayes

In [35]:
nb= NaiveBayes()

In [36]:
from pyspark.ml import Pipeline

In [37]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric,tokenizer,
                                 stop_remove,count_vec,idf,
                                 clean_up])

In [38]:
cleaner = data_prep_pipe.fit(data)

In [39]:
clean_data= cleaner.transform(data)

In [40]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [41]:
clean_data = clean_data.select('label', 'features')

In [42]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13459,[8,12,33,6...|
|  0.0|(13459,[0,26,308,...|
|  1.0|(13459,[2,14,20,3...|
|  0.0|(13459,[0,73,84,1...|
|  0.0|(13459,[36,39,140...|
|  1.0|(13459,[11,57,62,...|
|  0.0|(13459,[11,55,108...|
|  0.0|(13459,[133,195,4...|
|  1.0|(13459,[1,50,124,...|
|  1.0|(13459,[0,1,14,29...|
|  0.0|(13459,[5,19,36,4...|
|  1.0|(13459,[9,18,40,9...|
|  1.0|(13459,[14,32,50,...|
|  0.0|(13459,[42,99,101...|
|  0.0|(13459,[567,1745,...|
|  1.0|(13459,[32,113,11...|
|  0.0|(13459,[86,224,47...|
|  0.0|(13459,[0,2,52,13...|
|  0.0|(13459,[0,77,107,...|
|  1.0|(13459,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [43]:
training, test = clean_data.randomSplit([0.7,0.3])

In [44]:
spam_detector = nb.fit(training)

In [45]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [46]:
test_results = spam_detector.transform(test)

In [47]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13459,[0,1,2,8,9...|[-812.00627548118...|[1.0,1.7006563920...|       0.0|
|  0.0|(13459,[0,1,3,10,...|[-607.52614583565...|[1.0,2.9852245155...|       0.0|
|  0.0|(13459,[0,1,4,53,...|[-798.17966116375...|[1.0,2.1766170469...|       0.0|
|  0.0|(13459,[0,1,5,15,...|[-741.86019547027...|[1.0,3.3355968393...|       0.0|
|  0.0|(13459,[0,1,6,16,...|[-1004.4802657081...|[1.0,2.4588925398...|       0.0|
|  0.0|(13459,[0,1,8,9,1...|[-883.43333114660...|[1.0,1.4641305662...|       0.0|
|  0.0|(13459,[0,1,8,9,1...|[-1177.8758045511...|[1.0,3.7985730042...|       0.0|
|  0.0|(13459,[0,1,10,15...|[-537.88309491634...|[1.0,1.6084493891...|       0.0|
|  0.0|(13459,[0,1,13,35...|[-447.70974995883...|[1.0,1.2218551054...|       0.0|
|  0.0|(13459,[0

In [48]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [49]:
acc_eval = MulticlassClassificationEvaluator()

In [50]:
acc = acc_eval.evaluate(test_results)

In [51]:
print('ACC of NB Model')
print(acc)

ACC of NB Model
0.9258731294984092
